In [17]:
import os
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator
import librosa
import librosa.display
import matplotlib.pyplot as plt

import math
import glob

baseline = pickle.load(open('/home/ytang363/7100_voiceConversion/11-02_baseline-results.pkl', "rb"))
retrain = pickle.load(open('/home/ytang363/7100_voiceConversion/11-02_results.pkl', "rb"))

In [2]:
def log_spec_dB_dist(x, y):
    log_spec_dB_const = 10.0 / math.log(10.0) * math.sqrt(2.0)
    diff = x - y
    return log_spec_dB_const * math.sqrt(np.inner(diff, diff))

cost_function = log_spec_dB_dist

In [5]:
# Load ground truth utterance
rootDir = '/home/ytang363/7100_voiceConversion/VCTK-Corpus-0.92/spmel-16k'
dirName, subdirList, _ = next(os.walk(rootDir))
process_speakers = ['p225', 'p226', 'p227', 'p228', 'p229', 'p230', 'p231', 'p232', 'p233', 'p234', 'p237']
process_uttr = '001'
speakers = []
gt = {}

for speaker in sorted(subdirList):
    if speaker not in process_speakers:
        continue

    print('Processing speaker: %s' % speaker)
    _, _, fileList = next(os.walk(os.path.join(dirName,speaker)))

    indices = [i for i, element in enumerate(fileList) if process_uttr in element]
    print(fileList[indices[0]]) 

    spec = np.load(os.path.join(dirName, speaker, fileList[indices[0]]))

    gt[speaker] = spec

Processing speaker: p225
p225_001_mic1.npy
Processing speaker: p226
p226_001_mic1.npy
Processing speaker: p227
p227_001_mic1.npy
Processing speaker: p228
p228_001_mic1.npy
Processing speaker: p229
p229_001_mic1.npy
Processing speaker: p230
p230_001_mic1.npy
Processing speaker: p231
p231_001_mic1.npy
Processing speaker: p232
p232_001_mic1.npy
Processing speaker: p233
p233_001_mic1.npy
Processing speaker: p234
p234_001_mic1.npy
Processing speaker: p237
p237_001_mic1.npy


In [31]:
# Loop for evaluation

def avg_mcd(gen_mel):
    total_diff = 0
    for i in range(len(process_speakers)):
        current = gen_mel[i][0].split('x')[1]
        test_uttr = gen_mel[i][1] # for p225-001, the shape is (129,80)
        gt_uttr = gt[current]

        min_cost, _ = librosa.sequence.dtw(gt_uttr[:, 1:].T, test_uttr[:, 1:].T, 
                                            metric=cost_function)   
        diff = np.mean(min_cost) / len(gt)
        total_diff += diff
    avg_MCD = total_diff / len(process_speakers)
    return avg_MCD       

In [32]:
avg_mcd(baseline)

70.75646523216993

In [33]:
avg_mcd(retrain)

66.64690770370817